In [17]:
import torch
from torch import nn
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class Weight(nn.Module):
    def __init__(self):
        super(Weight, self).__init__()
        self.w = nn.Parameter(torch.tensor([0.5]))
        self.delta = nn.Parameter(torch.randn(8,8))
    def forward(self,x):
        output = x + self.delta[0]
        return output*self.w

class MLP(nn.Module):
    def __init__(self,feature_num , rule_num, center,sigma ):
        super(MLP, self).__init__()
        self.fc = nn.Linear(feature_num, 32)
        self.relu = nn.ReLU()
        self.drop_out= nn.Dropout(0.1)
        self.fc2 = nn.Linear(32, 16)
        self.relu2 = nn.ReLU()
        self.drop_out2= nn.Dropout(0.1)
        self.fc3 = nn.Linear(16, 8)
        self.weight = Weight()
    def forward(self, x):
        input = self.fc(x)
        input = self.relu(input)
        input = self.drop_out(input)
        input = self.fc2(input)
        input = self.relu2(input)
        input = self.drop_out2(input)
        output = self.fc3(input)
        return self.weight(output)

In [ ]:
features = torch.randn((100, 13))
target = torch.randint(0,5, (100,1))

mlp = MLP(13,10, "","")
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.0001, weight_decay=0)
criterion = nn.CrossEntropyLoss()
for i in range(100):
    x = torch.tensor(features[i])
    target = torch.tensor(target[i]).squeeze()
    output = mlp(x)
    loss = criterion(output, target)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    for name, parms in mlp.named_parameters():
        print('-->name:', name)
        # print('-->para:', parms)
        print('-->grad_requirs:',parms.requires_grad)
        print('-->grad_value:',parms.grad)

In [31]:
x = torch.tensor([0,1,2,3,4,5,6,7])
# x = x.view(1,-1)
print(x)
x_arr = x.repeat(1, 10).view(-1, 8)
print(x_arr)

tensor([0, 1, 2, 3, 4, 5, 6, 7])
tensor([[0, 1, 2, 3, 4, 5, 6, 7],
        [0, 1, 2, 3, 4, 5, 6, 7],
        [0, 1, 2, 3, 4, 5, 6, 7],
        [0, 1, 2, 3, 4, 5, 6, 7],
        [0, 1, 2, 3, 4, 5, 6, 7],
        [0, 1, 2, 3, 4, 5, 6, 7],
        [0, 1, 2, 3, 4, 5, 6, 7],
        [0, 1, 2, 3, 4, 5, 6, 7],
        [0, 1, 2, 3, 4, 5, 6, 7],
        [0, 1, 2, 3, 4, 5, 6, 7]])


In [2]:
import torch
rule = torch.tensor([1,2,3,4,5,6,7,8,9,10])
memory = torch.tensor([2,2,3,4,4,4,4,4,4,4])
recurrent_weight = torch.tensor([0.1,0.1, 0.1,0.1,0.1,0.1, 0.1,0.1,0.1,0.1])
recurrent_memory = torch.mul(memory, recurrent_weight)
print(recurrent_memory)
rule = torch.mul(rule, recurrent_memory)
print(rule)

tensor([0.2000, 0.2000, 0.3000, 0.4000, 0.4000, 0.4000, 0.4000, 0.4000, 0.4000,
        0.4000])
tensor([0.2000, 0.4000, 0.9000, 1.6000, 2.0000, 2.4000, 2.8000, 3.2000, 3.6000,
        4.0000])


In [15]:
from torch import nn
import torch
import torch.nn.functional as F

class MLP(nn.Module):
    def __init__(self,feature_num , rule_num):
        super(MLP, self).__init__()
        self.fc = nn.Linear(feature_num, 32)
        self.fc2 = nn.Linear(32, 16)
        self.relu = nn.ReLU()
        self.drop_out= nn.Dropout(0.1)
        self.fc3 = nn.Linear(16, 2)
    def forward(self, x):
        input = self.fc(x)
        input = self.fc2(input)
        input = self.relu(input)
        input = self.drop_out(input)
        output = self.fc3(input)
        return output

model = MLP(13, 10)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001,weight_decay=0)
criterion = nn.CrossEntropyLoss()
features= torch.randn((100, 13))
labels = torch.randint(0,1,(100,1))
epoch = 20
for i in range(epoch):
    count = 0
    total_loss = 0
    for i in range(100):
        optimizer.zero_grad()
        x = features[i]
        target = labels[i].squeeze()
        output= model(x)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        count = count+ 1
        total_loss =total_loss + loss.item()
        if count %10 ==0:
            print("epoch:",i, ",coutn:", count, ",loss:", total_loss/count)

epoch: 9 ,coutn: 10 ,loss: 0.7715085506439209
epoch: 19 ,coutn: 20 ,loss: 0.7626175701618194
epoch: 29 ,coutn: 30 ,loss: 0.780062468846639
epoch: 39 ,coutn: 40 ,loss: 0.7694632485508919
epoch: 49 ,coutn: 50 ,loss: 0.7649408495426178
epoch: 59 ,coutn: 60 ,loss: 0.7622565160195033
epoch: 69 ,coutn: 70 ,loss: 0.7642722879137311
epoch: 79 ,coutn: 80 ,loss: 0.7611402288079262
epoch: 89 ,coutn: 90 ,loss: 0.7602542943424648
epoch: 99 ,coutn: 100 ,loss: 0.7574954503774642
epoch: 9 ,coutn: 10 ,loss: 0.7392565667629242
epoch: 19 ,coutn: 20 ,loss: 0.721653938293457
epoch: 29 ,coutn: 30 ,loss: 0.7371583660443624
epoch: 39 ,coutn: 40 ,loss: 0.7278724819421768
epoch: 49 ,coutn: 50 ,loss: 0.7240020215511322
epoch: 59 ,coutn: 60 ,loss: 0.7199669847885768
epoch: 69 ,coutn: 70 ,loss: 0.7241948579038893
epoch: 79 ,coutn: 80 ,loss: 0.719625560939312
epoch: 89 ,coutn: 90 ,loss: 0.7207553519142998
epoch: 99 ,coutn: 100 ,loss: 0.7170013153553009
epoch: 9 ,coutn: 10 ,loss: 0.6854872584342957
epoch: 19 ,coutn: